In [ ]:
# Francois Kroll 19/05/2023
# this script is just some edits to Measure_activevoxels_masks_CT.ipynb written by Chintan Trivedi
# here goal is to measure total signal within masks (i.e. sum of gray intensity of every voxel in the mask)
# instead of counting voxels above a certain threshold

import os 
import numpy as np
from numpy import mean
from numpy import median
import skimage
import tifffile as tifffile
from skimage.transform import downscale_local_mean
from skimage.filters import rank_order
from skimage.filters import median
from skimage.filters import threshold_multiotsu as multiotsu
import csv
import nibabel as nib

: 

In [ ]:
import sys
print(sys.version)

In [2]:
# FK added 19/05/2023
# small function to avoid importing .DS_Store on macos
def listdir_nohidden(path):
    return( sorted([f for f in os.listdir(path) if not f.startswith('.')]) )

In [3]:
### main function which takes one brain and calculates, for each mask, sum, mean, median of every voxel within that mask


def mask_process(mask_path, brain):
    mask_pixels = []
    sum_pixels = []
    mean_pixels = []
    median_pixels = []
    max_pixels = []
    min_pixels = []
    
    for files in listdir_nohidden(mask_path):
        mask = tifffile.imread(mask_path + files) # open one mask at a time; # FK 19/05/2023 deleted + '//' + as using on macos

        mask_pixels.append( np.count_nonzero(mask) ) # number of pixels in mask
        sum_pixels.append( sum(brain[np.nonzero(mask)]) ) # sum of all pixels gray value
        mean_pixels.append( np.mean(brain[np.nonzero(mask)]) ) # mean of all pixels gray value
        median_pixels.append( np.median(brain[np.nonzero(mask)]) ) # median of all pixels gray value
        min_pixels.append( np.min(brain[np.nonzero(mask)]) ) # min of all pixels gray value
        max_pixels.append( np.max(brain[np.nonzero(mask)]) ) # max of all pixels gray value
        
    return mask_pixels, sum_pixels, mean_pixels, median_pixels, min_pixels, max_pixels

In [4]:
mask_dir = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/zbbMasks/" #mask directory

# – RUN1 –
# brain_dir = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/tphSCR/" #data directory

# – RUN2 –
# brain_dir = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/tphSORL1/" #data directory

# – RUN3 –
# brain_dir = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/sertSCR/" #data directory

# – RUN4 –
# brain_dir = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/sertSORL1/" #data directory


### chunk below prepares the list of brains ready to be analysed
brains=[]

for files in listdir_nohidden(brain_dir): # FK 19/05/2023 was os.listdit(brain_dir), changed to def listdir_nohidden(brain_dir): because on macos tries to import .DS_Store file
    brain = tifffile.imread(brain_dir + files) # FK 19/05/2023 deleted + '//' + as using on macos
    rescale_brain = downscale_local_mean(brain, (2, 2, 2)) # scale brains down to mask size
    rescale_brain = rescale_brain.astype('uint8') # FK no rescaling is occuring here (variable is rescale_brain), I think just conversion
    brains.append(rescale_brain)
    %xdel brain

In [5]:
# some useful stuff for debugging
# mask = tifffile.imread('/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/000_MASKS/Anterior commissure.tif')
# print(brain[np.nonzero(mask)])
# np.median(brain[np.nonzero(mask)])


In [6]:
pixel_mask_result = [] #list of dictionaries for each brain: {mask name: measures}
mask_names = listdir_nohidden(mask_dir)

i=1

for brain in brains:
    print('brain num: ', i)
    mask_pix, sum_pix, mean_pix, median_pix, min_pix, max_pix = mask_process(mask_dir, brain) # calling function to analyse pixels within every mask
    ziptogether = zip(mask_names, mask_pix, sum_pix, mean_pix, median_pix, min_pix, max_pix)
    pixel_mask_list = list(ziptogether)
    pixel_mask_result.append(pixel_mask_list)
    i=i+1

brain num:  1
brain num:  2
brain num:  3
brain num:  4
brain num:  5
brain num:  6
brain num:  7
brain num:  8
brain num:  9
brain num:  10


In [7]:
#change: NT_geno (neurotransmitter and genotype), writefile (save to folder)

writefile = "/Users/francoiskroll/Dropbox/ZFAD/sorl1HCR/intensityVoxels_v3/" # results file directory
# NT_geno = "SCR_tph" # suffix for control or treated or gene expression
# NT_geno = "SORL1_tph"
# NT_geno = "SCR_sert"
# NT_geno = "SORL1_sert"

brain_num = 1 # initialise brain number

headers = ['anatomy', 'numVx', 'sumVx', 'meanVx', 'medianVx', 'minVx', 'maxVx']

for pixels in pixel_mask_result:
    pix_data = open(writefile + NT_geno + str(brain_num) + '.csv', 'w', newline='') # FK 19/05/2023 deleted + '//' + as using on macos
    brain_num += 1
    writer = csv.writer(pix_data)
    writer.writerow(headers)
    
    for ind, data in enumerate(pixels):
        writer.writerow(data)
        
    pix_data.close()